In [1]:
import boto3
import xmltodict
import json
import pandas as pd
import pathlib
import collections
from datetime import datetime, timezone

In [2]:
create_hits_in_production = True
environments = {
  "production": {
    "endpoint": "https://mturk-requester.us-east-1.amazonaws.com",
    "preview": "https://www.mturk.com/mturk/preview"
  },
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["production"] if create_hits_in_production else environments["sandbox"]
session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

In [3]:
# This will return your current MTurk balance if you are connected to Production.
# If you are connected to the Sandbox it will return $10,000.
print(client.get_account_balance()['AvailableBalance'])

228.27


In [482]:
batch_df = pd.read_csv('../../data/batched/115hhrg30242_batch2_hits1.csv', index_col=False)
print(batch_df.shape)
batch_df

(1, 84)


,hearing_id,hearing_title,hearing_date,hearing_summary,turn1_id,q1_speaker,q1_speaker_detail,q1_text,r1_id,r1_speaker,...,r5_speaker_detail,r5_text,prev_turn5_id,prev_turn5_speaker,prev_turn5_speaker_detail,prev_turn5_text,next_turn5_id,next_turn5_speaker,next_turn5_speaker_detail,next_turn5_text
0,CHRG-115hhrg30242,Examining the Equifax Data Breach,10/5/17,Former Equifax Chair and CEO Richard Smith tes...,3,Chairman Hensarling,"politician: JEB HENSARLING, Texas, Chairman",It has not been the practice of the committee ...,4,Mr. Smith,...,"witness: Smith, Richard F., Adviser to the In...",Yes.,10,Mr. Smith,"witness: Smith, Richard F., Adviser to the In...","Yes. Our policy, our program at the time was w...",13,Chairman Hensarling,"politician: JEB HENSARLING, Texas, Chairman",So what happened?


In [483]:
html_layout = open('./mturk_hit_intent.html', 'r').read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)

In [484]:
TaskAttributes = {
    'MaxAssignments': 1,     
    # How long the task will be available on the MTurk website (1 week)
    'LifetimeInSeconds': 60*60*24*7, 
    # How long before automatically approving workers (3 days)
    'AutoApprovalDelayInSeconds': 60*60*24*3, 
    # How long Workers have to complete each item (1 hour)
    'AssignmentDurationInSeconds': 60*60, 
    # The reward you will offer Workers for each response
    'Reward': '1.00',     
    'Title': 'What is the intent behind questions and responses in U.S. congressional hearings?',
    'Keywords': 'dialogue, politics, subjective, question, answer, intent',
    'Description': 'Infer the perhaps subversive intent of questions and responses for witnesses and congressional members in U.S congressional hearings.'
#     'Title': 'BONUS Opportunity: What is the intent behind questions and responses in U.S. congressional hearings?',
#     'Keywords': 'bonus, dialogue, politics, subjective, question, answer, intent',
#     'Description': 'Infer the perhaps subversive intent of questions and responses for witnesses and congressional members in U.S congressional hearings. Receive a bonus to double your reward for quality HITs.'
}

In [485]:
column_names = list(batch_df.columns)
column_to_xml = {}
for column_name in column_names:
    column_to_xml[column_name] = '${'+column_name+'}'

for k,v in list(column_to_xml.items()):
    if v not in question_xml:
        print('Warning: Ignoring value: ', v)
        column_to_xml.pop(k)

In [486]:
"""
Create hits

"""
#batch2_115hhrg25545_results = []
batch2_115hhrg30956_results = []
for hit in batch_df.itertuples():
    # Convert named tuple to dictionary
    question_xml_filled = question_xml
    hit_dict = hit._asdict()
    for key in hit_dict.keys():
        if key in column_to_xml:
            question_xml_filled = question_xml_filled.replace(column_to_xml[key],str(hit_dict[key]))
    response = client.create_hit(
       **TaskAttributes,
       Question=(question_xml_filled),
        QualificationRequirements=[
            {
                'QualificationTypeId': '3BERVBF4P91CXVY7ZYVLFJAE0K7YPS', #'3D39V6MAT4NEO0YLHPBLW6N58MO8C9', '3PHD6OHA876NG3N038H08WPJK8Q9WA', '3O3C52ZQESTNOSOAHFVROOKMX83HSU', 
                'Comparator' : 'Exists',
                'ActionsGuarded': 'DiscoverPreviewAndAccept'
            },
#             {
#                 'QualificationTypeId': '3ZLV9K6K8DM9XJZU0NZ4HDNIGPKBA5',
#                 'Comparator' : 'DoesNotExist',
#                 'ActionsGuarded': 'DiscoverPreviewAndAccept'
#             },
            {
                'QualificationTypeId': '000000000000000000L0',
                'Comparator': 'GreaterThanOrEqualTo',
                'IntegerValues': [
                    96,
                ],
                'RequiredToPreview': True,
                'ActionsGuarded': 'DiscoverPreviewAndAccept'
            },
            {
                'QualificationTypeId': '00000000000000000071',
                'Comparator': 'EqualTo',
                'LocaleValues': [
                    {
                        'Country': 'US'
                    }
                ],
                'RequiredToPreview': True,
                'ActionsGuarded': 'DiscoverPreviewAndAccept'
            },
            {
                'QualificationTypeId': '00000000000000000040',
                'Comparator': 'GreaterThanOrEqualTo',
                'IntegerValues': [
                    500,
                ],
                'RequiredToPreview': True,
                'ActionsGuarded': 'DiscoverPreviewAndAccept'
            },
        ]        
     )
    batch2_115hhrg30956_hit_type_id = response['HIT']['HITTypeId']
    hit_dict['hit_id'] = response['HIT']['HITId']
    batch2_115hhrg30956_results.append(hit_dict)
print("Created ", len(batch2_115hhrg30956_results), " hits.")   
print("You can view the HITs here:")
print(mturk_environment['preview']+"?groupId={}".format(batch2_115hhrg30956_hit_type_id))

Created  1  hits.
You can view the HITs here:
https://www.mturk.com/mturk/preview?groupId=3TKAYVZVPKLSTMSUNCADSYMAR0J82F


## Get Results

In [4]:
def get_true(answers_dict, lookup_key):
    if lookup_key in answers_dict:
        my_dict = answers_dict[lookup_key]
        for key in my_dict.keys():
            if my_dict[key]:
                return key
    return None

def get_trues(answers_dict, sub_dict_keys, main_dict):
    key = None
    for sub_dict_key in sub_dict_keys:
        if sub_dict_key in answers_dict:
            sub_dict = answers_dict[sub_dict_key]
            key = get_true(sub_dict)
            if key:
                break
    if key:
        return key
    else:
        return get_true(main_dict)

def get_q_intent(answers_dict, q_loaded_key, q_favor_key):
    q_loaded = get_true(answers_dict, q_loaded_key)
    if (q_loaded == "yesLoad"):
        return "attack"
    else:
        q_favor = get_true(answers_dict, q_favor_key)
        if (q_favor == "yesFavor"):
            return "favor"
        else:
            return "neutral"

In [5]:
def get_hit_data(results, has_opposing_intent=True):
    data = {}
    for item in results: 
        # Get the status of the HIT
        hit = client.get_hit(HITId=item['hit_id'])
        item['status'] = hit['HIT']['HITStatus']
        assignmentsList = client.list_assignments_for_hit(
            HITId=item['hit_id'],

        )

        assignments = assignmentsList['Assignments']
        item['assignments_submitted_count'] = len(assignments)
        answers = []

        # register HIT
        data[item['hit_id']] = {}
        data[item['hit_id']]['hearing_id'] = item['hearing_id']
        data[item['hit_id']]['hearing_title'] = item['hearing_title']
        data[item['hit_id']]['qa1'] = {'q_id': item['turn1_id'], 'r_id': item['r1_id'], 'answers': []}
        data[item['hit_id']]['qa2'] = {'q_id': item['turn2_id'], 'r_id': item['r2_id'], 'answers': []}
        data[item['hit_id']]['qa3'] = {'q_id': item['turn3_id'], 'r_id': item['r3_id'], 'answers': []}
        data[item['hit_id']]['qa4'] = {'q_id': item['turn4_id'], 'r_id': item['r4_id'], 'answers': []}
        data[item['hit_id']]['qa5'] = {'q_id': item['turn5_id'], 'r_id': item['r5_id'], 'answers': []}

        for assignment in assignments:
            # Retrieve the attributes for each Assignment
            worker_id = assignment['WorkerId']
            work_time_seconds = (assignment['SubmitTime'] - assignment['AcceptTime']).total_seconds()
            submit_time = assignment['SubmitTime'].isoformat()
            assignment_id = assignment['AssignmentId']

            # Retrieve the value submitted by the Worker from the XML
            xml_doc = xmltodict.parse(assignment['Answer'])
            answers_dict = json.loads(xml_doc['QuestionFormAnswers']['Answer']['FreeText'])[0]
            answers.append(answers_dict)

            sent_q_key = 'sentimentQuestioner'
            sent_r_key = 'sentimentWitness'
            for idx in range(1,5+1):
                q_loaded_key = f'qLoad{idx}'
                q_favor_key = f'qFavor{idx}'
                r_answer_key = f'r{idx}'
                r_overanswer_key = f'rOverans{idx}'
                r_intent_part_key = f'rPart{idx}'
                r_intent_shift_key = f'rShift{idx}'
                r_intent_cantanswer_key = f'rCantAnswer{idx}'
                r_intent_overanswer_key = f'rIntentOverans{idx}'

                if has_opposing_intent:
                    r_answer_opp_key = f'rOpp{idx}'
                    r_overanswer_opp_key = f'rOppOverans{idx}'
                    r_intent_opp_overanswer_key = f'rOppIntentOverans{idx}'
                    r_intent_opp_part_key = f'rOppPart{idx}'
                    r_intent_opp_shift_key = f'rOppShift{idx}'
                    r_intent_opp_cantanswer_key = f'rOppCantAnswer{idx}'

                explain_intent_key = f'explainIntent{idx}'
                if has_opposing_intent:
                    explain_intent_opp_key = f'explainOpposingIntent{idx}'
                
                if has_opposing_intent:
                    info = {
                            'assignment_id': assignment_id,
                            'worker_id': worker_id,
                            'work_time_secs': work_time_seconds,
                            'submit_time': submit_time,
                            'sentiment_q': get_true(answers_dict, sent_q_key),
                            'sentiment_r': get_true(answers_dict, sent_r_key),
                            'q_intent': get_q_intent(answers_dict, q_loaded_key, q_favor_key),
                            'r_answer': get_true(answers_dict, r_answer_key),
                            'r_overanswer': get_true(answers_dict, r_overanswer_key),
                            'r_intent_part': get_true(answers_dict, r_intent_part_key),
                            'r_intent_shift': get_true(answers_dict, r_intent_shift_key),
                            'r_intent_cantanswer': get_true(answers_dict, r_intent_cantanswer_key),
                            'r_intent_overanswer': get_true(answers_dict, r_intent_overanswer_key),
                            'r_answer_opposing': get_true(answers_dict, r_answer_opp_key),
                            'r_overanswer_opposing': get_true(answers_dict, r_overanswer_opp_key),
                            'r_intent_opp_part': get_true(answers_dict, r_intent_opp_part_key),
                            'r_intent_opp_shift': get_true(answers_dict, r_intent_opp_shift_key),
                            'r_intent_opp_cantanswer': get_true(answers_dict, r_intent_opp_cantanswer_key),
                            'r_intent_opp_overanswer': get_true(answers_dict, r_intent_opp_overanswer_key),
                            'r_explain_intent': answers_dict[explain_intent_key],
                            'r_explain_intent_opposing': answers_dict[explain_intent_opp_key]
                    }
                else:
                    info = {
                            'assignment_id': assignment_id,
                            'worker_id': worker_id,
                            'work_time_secs': work_time_seconds,
                            'submit_time': submit_time,
                            'sentiment_q': get_true(answers_dict, sent_q_key),
                            'sentiment_r': get_true(answers_dict, sent_r_key),
                            'q_intent': get_q_intent(answers_dict, q_loaded_key, q_favor_key),
                            'r_answer': get_true(answers_dict, r_answer_key),
                            'r_overanswer': get_true(answers_dict, r_overanswer_key),
                            'r_intent_part': get_true(answers_dict, r_intent_part_key),
                            'r_intent_shift': get_true(answers_dict, r_intent_shift_key),
                            'r_intent_cantanswer': get_true(answers_dict, r_intent_cantanswer_key),
                            'r_intent_overanswer': get_true(answers_dict, r_intent_overanswer_key),
                            'r_explain_intent': answers_dict[explain_intent_key]
                    }
                qa_key = f'qa{idx}'
                data[item['hit_id']][qa_key]['answers'].append(info)

        # Add the answers that have been retrieved for this item
        item['answers'] = answers
    return(data)

### check assignment status

In [6]:
def save_batches(batch_dict, save_opposing_intent):
    for batch_date, base_name in batch_dict.items():
        save_batch(batch_date, base_name, save_opposing_intent)
        
def save_batch(batch_date, base_name, save_opposing_intent):
    if 'raters' in base_name:
        batch_file = base_name[:(base_name.find('raters')-1)] + '.csv'
        save_file = base_name + '_' + batch_date + '.json'
        batch_df = pd.read_csv('../../data/batched/'+batch_file, index_col=False)

        hit_ids = batch_to_hit_ids[batch_date]
        results = []
        for idx,hit in enumerate(batch_df.itertuples()):
            hit_dict = hit._asdict()
            hit_dict['hit_id'] = hit_ids[idx]
            results.append(hit_dict)
        data = get_hit_data(results, has_opposing_intent=save_opposing_intent)
        with open('../../data/annotated/'+save_file,'w+') as f:
            f.write(json.dumps(data))
        print("Finished saving", batch_date)
    else:
        print("Skipping batch", batch_date)
        

In [7]:
batch_to_hit_ids = collections.defaultdict(list)

#start_date_string = "2019-11-20"
start_date_string = "2020-03-22"
#start_date_string = "2020-02-07"
start_date = datetime.strptime(start_date_string, "%Y-%m-%d").date()
hit_title = "What is the intent"

# # get all hits
hits=[]
paginator = client.get_paginator('list_hits')
i=0
for page in paginator.paginate():
    hits.extend(page['HITs'])
    i += 1
    if i > 5: #225
        break;

prev_creation_time = None       
for item in hits:
        hit_id=item['HITId']
        creation_time = client.get_hit(HITId=hit_id)['HIT']['CreationTime']
        delta = creation_time.date() - start_date
        if hit_title in item['Title'] and delta.days >= 0:
            delta_hits = None
            if prev_creation_time:
                delta_hits = prev_creation_time - creation_time
            if delta_hits and delta_hits.days == 0 and delta_hits.seconds <= 100:
                creation_time_str = prev_creation_time.strftime('%m-%d-%Y_%Hh%Mm')
            else:
                creation_time_str = creation_time.strftime('%m-%d-%Y_%Hh%Mm')
                prev_creation_time = creation_time
            batch_to_hit_ids[creation_time_str].append(hit_id)
            print(creation_time_str, hit_id, item['HITStatus'])
        
        

03-22-2020_06h56m 3CZH926SICD0F99PA1ALT8M4OFF4E5 Reviewable
03-22-2020_06h39m 38VTL6WC4ACGHXUGO75P53KY6NEY5I Reviewable
03-22-2020_06h39m 3QQUBC64ZEDV7HACNBF9G6DBV2KNXR Reviewable
03-22-2020_06h39m 33QQ60S6ASH05HOAH02OLNIUTRT0U2 Reviewable
03-22-2020_06h39m 3EQVJH0T40KWG4FHF3JZMJV9WUNTH5 Reviewable
03-22-2020_06h39m 3OLZC0DJ8JEBPRUQQG1ZNDK9TSIVIG Reviewable
03-22-2020_06h39m 3QO7EE372OM9XBK3Y7F8V3HOHAJQB8 Reviewable
03-22-2020_06h39m 3G3AJKPCXLRYLEQGA2MF0BEK6RJY4D Reviewable
03-22-2020_06h39m 3K8CQCU3KE0GIH5MD6JNENZ4M0ENWY Reviewable
03-22-2020_06h39m 3DWNFENNE3U8QPVHO2LCQOZWTNAJ44 Reviewable


In [8]:
for hits in batch_to_hit_ids.values():
    hits.reverse()

In [9]:
#remove empty hit from sessions batch
#batch_to_hit_ids['12-10-2019_13h16m']=['3CVDZS288HZ8PDYRBGUNXKO3EQ9MFM']

In [10]:
for batch_date, hit_ids in batch_to_hit_ids.items():
    print('\n')
    print("Batch date: ", batch_date)
    num_hits = len(hit_ids)
    num_raters_per_hit = []
    complete_times = []
    expiration_times = []
    for hit_id in hit_ids:
        print(hit_id)
        hit = client.get_hit(HITId=hit_id)['HIT']
        summary_idx = hit['Question'].index('Summary:')
        first_hit_idx = hit['Question'].index('q1Spkr')
        print(hit['Question'][summary_idx:summary_idx+80])
        print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
        assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
        num_raters_per_hit.append(len(assignments))
        complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
        expiration_times.append(hit['Expiration'].date())
    num_raters = hit['MaxAssignments']
    print("Num hits: ", num_hits)
    print("Num raters: ", num_raters)
    print("Raters per assignment: ", num_raters_per_hit)
    print("Completion times: ", complete_times)
    print("Expiration times: ", expiration_times)



Batch date:  03-22-2020_06h56m
3CZH926SICD0F99PA1ALT8M4OFF4E5
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
[politician: JEB HENSARLING, Texas, Chairman]</span>: It has not been the practice of the committee 
Num hits:  1
Num raters:  1
Raters per assignment:  [1]
Completion times:  [[datetime.date(2020, 3, 22)]]
Expiration times:  [datetime.date(2020, 3, 29)]


Batch date:  03-22-2020_06h39m
3DWNFENNE3U8QPVHO2LCQOZWTNAJ44
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
[politician: JEB HENSARLING, Texas, Chairman]</span>: So what happened?
				</td>
				<th class="lbo
3K8CQCU3KE0GIH5MD6JNENZ4M0ENWY
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
an: MAXINE WATERS, California, Ranking]</span>: You are not paid. And so you came today to try and p
3G3AJKPCXLRYLEQGA2MF0BEK6RJY4D
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
itician: BLAINE LUETKEMEYER, 

In [11]:
# Fix batch order
def swap_hit_order(batch_date, hit1_id, hit2_id):
    hits = batch_to_hit_ids[batch_date]
    hit1_idx = hits.index(hit1_id)
    hit2_idx = hits.index(hit2_id)
    hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
    batch_to_hit_ids[batch_date] = hits
    for hit_id in batch_to_hit_ids[batch_date]:
            print(hit_id)
            hit = client.get_hit(HITId=hit_id)['HIT']
            summary_idx = hit['Question'].index('Summary:')
            first_hit_idx = hit['Question'].index('q1Spkr')
            print(hit['Question'][summary_idx:summary_idx+80])
            print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
            assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
            num_raters_per_hit.append(len(assignments))
            complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
            expiration_times.append(hit['Expiration'].date())  

# batch_date = '03-13-2020_05h49m'
# hit1_id = '3WYZV0QBFJCT68LFKLA0PBDHDSUXBT'
# hit2_id = '3ULIZ0H1VA4JRRJDCIYGRG9OYHU519'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())

# batch_date = '03-12-2020_17h41m'
# hit1_id = '3X878VYTIEHP4AR47ASYE2REQWN7F4'
# hit2_id = '3J94SKDEKIODJMZPSBIBSIAXNDC5DC'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())
        
# batch_date = '03-09-2020_05h01m'
# hit1_id = '3IV1AEQ4DRC99QIJ05BP70F1UBLJ8T'
# hit2_id = '3DW3BNF1GHHI0LIDHUP8AOQVPYS8V3'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())

# batch_date = '03-06-2020_09h30m'
# hit1_id = '3P0I4CQYVY6Y025YELJ0TCZ9BYRWO5'
# hit2_id = '3RQVKZ7ZRKI53GFSJFDRUJXPH2N274'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())
        
# batch_date = '02-21-2020_16h43m'
# hit1_id = '3Q9SPIIRWJL9YYBX42EISHF3K73AWC'
# hit2_id = '3D42WVSDH8U8JUN5UJIYRFBASVGFYT'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())

# batch_date = '02-24-2020_06h19m'
# hit1_id = '3EQPA8A374VOTMJ69VB2KKC4R1EJZE'
# hit2_id = '31MBOZ6PAOQHGJWWK237P1OBM8XCLA'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]

# hit1_id = '32FESTC2NHPKZ9JQTGV8GOKUZZQUCM'
# hit2_id = '3U18MJKL1UL74JZHWFJM2TS0R3XCNC'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]

# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())
        

# batch_date = '02-25-2020_13h23m'
# hit1_id = '3PEG1BH7AEQNM8M8L2VBZH5EGRSKBH'
# hit2_id = '3P7RGTLO6ECI3YHGFOVF46L8Y97KAM'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())

# batch_date = '01-16-2020_16h18m'
# hit1_id = '3SU800BH86ROTAT9YYW30T9B16DUQV'
# hit2_id = '3J9L0X0VDFLVTF1SCW0IWM4JCML9WO'
# hits = batch_to_hit_ids[batch_date]
# hit1_idx = hits.index(hit1_id)
# hit2_idx = hits.index(hit2_id)
# hits[hit2_idx], hits[hit1_idx] = hits[hit1_idx], hits[hit2_idx]
# batch_to_hit_ids[batch_date] = hits
# for hit_id in batch_to_hit_ids[batch_date]:
#         print(hit_id)
#         hit = client.get_hit(HITId=hit_id)['HIT']
#         summary_idx = hit['Question'].index('Summary:')
#         first_hit_idx = hit['Question'].index('q1Spkr')
#         print(hit['Question'][summary_idx:summary_idx+80])
#         print(hit['Question'][first_hit_idx+80:first_hit_idx+180])
#         assignments = client.list_assignments_for_hit(HITId=hit_id)['Assignments']
#         num_raters_per_hit.append(len(assignments))
#         complete_times.append([assignment['SubmitTime'].date() for assignment in assignments])
#         expiration_times.append(hit['Expiration'].date())

In [12]:
swap_hit_order('03-22-2020_06h39m', '3G3AJKPCXLRYLEQGA2MF0BEK6RJY4D', '3QO7EE372OM9XBK3Y7F8V3HOHAJQB8')
swap_hit_order('03-22-2020_06h39m', '3EQVJH0T40KWG4FHF3JZMJV9WUNTH5', '33QQ60S6ASH05HOAH02OLNIUTRT0U2')
swap_hit_order('03-22-2020_06h39m', '3QQUBC64ZEDV7HACNBF9G6DBV2KNXR', '38VTL6WC4ACGHXUGO75P53KY6NEY5I')

3DWNFENNE3U8QPVHO2LCQOZWTNAJ44
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
[politician: JEB HENSARLING, Texas, Chairman]</span>: So what happened?
				</td>
				<th class="lbo
3K8CQCU3KE0GIH5MD6JNENZ4M0ENWY
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
an: MAXINE WATERS, California, Ranking]</span>: You are not paid. And so you came today to try and p
3QO7EE372OM9XBK3Y7F8V3HOHAJQB8
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
an: MAXINE WATERS, California, Ranking]</span>: How many?
				</td>
				<th class="lbordernowrap" bg
3G3AJKPCXLRYLEQGA2MF0BEK6RJY4D
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
itician: BLAINE LUETKEMEYER, Missouri]</span>: OK. Have you put in place or are you aware of new pro
3OLZC0DJ8JEBPRUQQG1ZNDK9TSIVIG
Summary:</strong> Former Equifax Chair and CEO Richard Smith testified before th
cian: CAROLYN B. MALONEY, New York]<

In [309]:
date_to_batch_name_opp = {
'11-20-2019_23h13m':'pilot_batch1_hits6_raters5',
'12-09-2019_18h51m':'pilot_batch1_hits6_raters9',
'12-10-2019_13h16m':'pilot_batch1sessions_hits1_raters1',
'12-15-2019_14h25m':'pilot_batch1_hits6_raters5',
'01-16-2020_16h18m':'pilot_batch2_hits6_raters7',
'02-03-2020_21h01m':'pilot_batch3_hits6_raters7'}
date_to_batch_name = {
'02-10-2020_14h07m':'pilot_batch4_hits4_raters3',
'02-10-2020_14h12m':'115hhrg31349_batch1_hits3_raters10',
'02-12-2020_22h26m':'115hhrg25545_batch1_hits3_raters7',
'02-13-2020_16h55m':'115hhrg25545_batch1_hits3_raters3',
'02-15-2020_05h36m':'115hhrg25545_batch2_hits7_raters7',
'02-17-2020_06h33m':'114hhrg26003_batch3_hits3_raters7',
'02-18-2020_19h55m':'114hhrg26003_batch4_hits4_raters7',
'02-19-2020_19h51m':'115hhrg30956_batch1_hits4_raters7',
'02-20-2020_11h09m':'pilot_batch5_hits6_raters4',
'02-20-2020_19h04m':'115hhrg30956_batch2_hits4_raters7',
'02-21-2020_16h43m':'113hhrg86195_batch1_hits8_raters7',
'02-24-2020_06h19m':'113hhrg89598_batch1_hits9_raters7',
'02-24-2020_15h13m':'115hhrg31349_batch1_hits1_raters1',
'02-24-2020_15h22m':'115hhrg31349_batch2_hits1_raters2',
'02-24-2020_17h41m':'115hhrg31349_batch3_hits1_raters2',
'02-24-2020_17h53m':'115hhrg31349_batch4_hits7_raters7',
'02-25-2020_13h23m':'115hhrg32380_batch1_hits9_raters7',
'02-26-2020_21h38m':'116hhrg36001_batch1_hits2_raters7',
'02-26-2020_22h24m':'116hhrg36001_batch2_hits7_raters7',
'02-27-2020_19h17m':'115hhrg31417_batch1Sandbox_hits6_raters4',
'02-27-2020_19h34m':'115hhrg31417_batch2_hits4_raters7',
'02-28-2020_12h37m':'113hhrg88494_batch1_hits1_raters1',
'02-28-2020_12h40m':'113hhrg88494_batch2_hits8_raters7',
'02-29-2020_14h32m':'113hhrg93834_batch1_hits2_raters7',
'03-03-2020_13h44m':'skip',
'03-03-2020_13h50m':'113hhrg93834_batch2_hits7_raters7',
'03-03-2020_16h20m':'115hhrg31349_batch2_hits1_raters2',
'03-03-2020_16h38m':'114hhrg20722_batch1_hits10_raters7',
'03-06-2020_09h30m':'114hhrg95063_batch1_hits9_raters7',
'03-08-2020_08h47m':'115hhrg31504_batch1_hits9_raters7',
'03-09-2020_05h01m':'116hhrg35230_batch1_hits9_raters7',
'03-10-2020_06h00m':'116hhrg37282_batch1_hits9_raters7',
'03-11-2020_13h38m':'113hhrg93834_batch3_hits1_raters1',
'03-11-2020_14h04m':'114hhrg20722_batch2_hits1_raters1',
'03-11-2020_21h15m':'114hhrg20722_batch3_hits1_raters2',
'03-11-2020_21h37m':'113hhrg93834_batch3_hits1_raters1',
'03-11-2020_21h41m':'116hhrg35589_batch1_hits10_raters7',
'03-12-2020_17h41m':'114hhrg22125_batch1_hits10_raters7',
'03-13-2020_05h49m':'114hhrg97630_batch1_hits10_raters7',
'03-17-2020_05h03m': '116hhrg37282_batch2_hits1_raters1'}

In [13]:
save_batch('03-22-2020_06h39m', '115hhrg30242_batch1_hits9_raters7', False)

Finished saving 03-22-2020_06h39m


In [ ]:
len(date_to_batch_name) + len(date_to_batch_name_opp)

In [ ]:
save_batches(date_to_batch_name_opp, True)

In [ ]:
save_batches(date_to_batch_name, False)

### expire hits

In [127]:
hit_ids = ['3Z56AA6EK4ZC6K5GANRJ3EBRSG5M65']
for hit_id in hit_ids:
    # Get the status of the HIT
    hit = client.get_hit(HITId=hit_id)
    status = hit['HIT']['HITStatus']
    print('HIT: ', hit_id, ', HITStatus:', status)

    if status=='Assignable':
        response = client.update_expiration_for_hit(
                HITId=item,
                ExpireAt=datetime(2020, 1, 1)
        )
        print(response)

ID:  {'HITId': '3RWO3EJELH8NK11WWX7UPME5P0WP1I', 'HITTypeId': '3IPH7GEBVFIET7MBDUMQ046J2QA1IZ', 'HITGroupId': '329HSLC9XEWTGZRND8U2XI2YLZ8HGG', 'CreationTime': datetime.datetime(2020, 3, 8, 8, 46, 53, tzinfo=tzlocal()), 'Title': 'What is the intent behind questions and responses in U.S. congressional hearings?', 'Description': 'Infer the perhaps subversive intent of questions and responses for witnesses and congressional members in U.S congressional hearings.', 'Question': '<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">\n        <HTMLContent><![CDATA[<!DOCTYPE html>\n<body>\n<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>\n<crowd-form>\n  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">\n  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js"></script>\n  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/js/

ParamValidationError: Parameter validation failed:
Invalid type for parameter HITId, value: {'HITId': '3RWO3EJELH8NK11WWX7UPME5P0WP1I', 'HITTypeId': '3IPH7GEBVFIET7MBDUMQ046J2QA1IZ', 'HITGroupId': '329HSLC9XEWTGZRND8U2XI2YLZ8HGG', 'CreationTime': datetime.datetime(2020, 3, 8, 8, 46, 53, tzinfo=tzlocal()), 'Title': 'What is the intent behind questions and responses in U.S. congressional hearings?', 'Description': 'Infer the perhaps subversive intent of questions and responses for witnesses and congressional members in U.S congressional hearings.', 'Question': '<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">\n        <HTMLContent><![CDATA[<!DOCTYPE html>\n<body>\n<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>\n<crowd-form>\n  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">\n  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquer

### delete hits

In [ ]:
3H6W48L9F4OGL2HZTLWFIC4YKP9WPD
3QREJ3J433WZZH8KCFK5DK1JS8ZKL7

In [ ]:
hits = ['3H6W48L9F4OGL2HZTLWFIC4YKP9WPD', '3QREJ3J433WZZH8KCFK5DK1JS8ZKL7']
for item in hits: 
    # Get the status of the HIT
    hit = client.get_hit(HITId=item)
    status = hit['HIT']['HITStatus']
    print('ID: ', item, ', HITStatus:', status)

    if status=='Assignable':
        response = client.update_expiration_for_hit(
                HITId=item,
                ExpireAt=datetime(2015, 1, 1)
        )
        print(response)

from datetime import datetime

# get all HITS
hits = []
paginator = client.get_paginator('list_hits')
i=0
for page in paginator.paginate():
    hits.extend(page['HITs'])
    i += 1
    if i > 3:
        break;

# delete
#    for item in client.list_hits()['HITs']:
for item in hits:
        hit_id=item['HITId']
        print('HITId:', hit_id)

        hit_type_id = item['HITTypeId']
        # Get HIT status
        status=client.get_hit(HITId=hit_id)['HIT']['HITStatus']
        print('HITStatus:', status)

        if hit_type_id=='3RSP1MSSW0FL8Z3XFLDCU56VFVAKUS':
            # If HIT is active then set it to expire immediately
            if status=='Assignable':
                response = client.update_expiration_for_hit(
                    HITId=hit_id,
                    ExpireAt=datetime(2015, 1, 1)
                )        
            if status=='Reviewable':
                assignmentsList = client.list_assignments_for_hit(
                    HITId=hit_id)
                assignments = assignmentsList['Assignments']
                for assignment in assignments:
                    ass_id = assignment['AssignmentId']
                    client.approve_assignment(AssignmentId=ass_id)
            # Delete the HIT
            try:
                client.delete_hit(HITId=hit_id)
            except:
                print('Not deleted')
            else:
                print('Deleted')

### Create Qualification for workers who participated in old pilot

In [ ]:
#11_01
pilot_workers_11_01 = ['A002160837SWJFPIAI7L7',
 'A17F06M11XQX6S',
 'A17TKHT8FEVH0R',
 'A18AP8I010IFW1',
 'A1A1ZXUVSV4YR8',
 'A1C0H8G0YI15MN',
 'A1GKD3NG1NNHRP',
 'A1QT7BQZVOBSY0',
 'A248IBH3PEK4MP',
 'A24LB89P1BPKKF',
 'A26RO8GGTQAXGG',
 'A28HB7240OFGEW',
 'A2E3TO92MCQ9XU',
 'A311A0TCGNTFKK',
 'A34Q9JGU37OQJM',
 'A3D8O2MXLJNVHC',
 'A3FAH5S2RGZ5AK',
 'A3HSONMX6LNZR4',
 'A3NMQ3019X6YE0',
 'A8YLQGRCTEKPR',
 'AMNFCZY7X063T',
 'AO33H4GL9KZX9',
 'AUZPUFWHNNTX5',
 'AYS5D72DT1WNT']
pilot_workers_11_13 = ['A1S8AP9P3NYZ6P',
 'A24KGCLUZK6KGT',
 'AUZPUFWHNNTX5',
 'A19ZA4GG7S0BKI',
 'A2JSIDRTDGDC7T',
 'A2HX1MQAFJZ6KK',
 'A14JGBZ7KHDI6C',
 'A53S7J4JGWG38',
 'A36ESW7AECVHK3',
 'A8CHA5VSC5KJR',
 'ALMAI0EOCZWNM',
 'A2R182TXQ26ZGE',
 'A2ALZGXEZFS8LC',
 'A1EJ4HA4K85PBJ',
 'A2XQ3P6M9NZNYB',
 'A1KEUUSB2OVIKC',
 'ALQEBL8XKDCWC',
 'A1LII4M7CS6C9I',
 'A30HUZHJBOX1LK',
 'A9MGD6PRW1N7K',
 'A20EU0UH5K3VGE',
 'A3B8DP6KHMP3OS',
 'A255C5NDOBXZEC']

pilot_workers_11_20 = ['AHSXMZBNS9BPA', 'A2JSIDRTDGDC7T', 'AK1GE19RT3PK6',
       'A38IPIPA3T3G4', 'A1CY7IOJ9YH136', 'A3141EW4WZT4Q0',
       'A3W1M37EJKYIZI', 'A1815TOQ1PZ7JH', 'A35IX4EYGXXFYR',
       'A1QXBYE81CP9N9', 'A17GP9RERC723H', 'A3RSYBAGLACUSB',
       'A2F2DDH12YU4AK', 'A2IMAGGCST8170', 'A24KGCLUZK6KGT',
       'A23FEELMFHLC9K', 'A3PLJLUKN3PEA2', 'A1GWYEIX5S6JU4',
       'A28TXHXPVNZ0KN', 'A35QZFRG4FKK3Z', 'A2P8V5SKYLL5I4',
       'A30HUZHJBOX1LK', 'A2R182TXQ26ZGE', 'A2DZ1QW6G1LUVY',
       'A3KPZRJSJL2NDB']
old_pilot_workers = pilot_workers_11_01 + pilot_workers_11_13 + pilot_workers_11_20

In [ ]:
response = client.create_qualification_type(
    Name='Completed Old Intent Pilot',
    Keywords='pilot',
    Description='Completed Old Intent Pilot',
    QualificationTypeStatus='Active',
    AutoGranted=False
)
print(response)

In [ ]:
def get_files(search_dir, suffix=None):
    glob_pattern = '*'
    if suffix:
        glob_pattern += suffix
    return [filepath.absolute() for filepath in pathlib.Path(search_dir).glob(glob_pattern)]

In [ ]:
# read all worker files
old_workers = []
worker_files = get_files('.', suffix='_workers.txt')
for worker_file in worker_files:
    with open(worker_file, 'r') as f:
        old_workers.extend(f.read().splitlines())

In [ ]:
old_workers = set(old_workers)
len(old_workers)

In [ ]:
for worker in old_workers:
    response_ass = client.associate_qualification_with_worker(
        QualificationTypeId='3ZLV9K6K8DM9XJZU0NZ4HDNIGPKBA5',
        WorkerId=worker,
        IntegerValue=1,
        SendNotification=False
    )
    print(response_ass)

## Create qualification for workers of specific HIT


### Qual 1

In [ ]:
response = client.create_qualification_type(
    Name='Re-annotate old HIT',
    Keywords='pilot',
    Description='Re-annotate old HIT',
    QualificationTypeStatus='Active',
    AutoGranted=False
)
print(response)

In [476]:
quals = client.list_workers_with_qualification_type(QualificationTypeId='3D39V6MAT4NEO0YLHPBLW6N58MO8C9',
    Status='Granted',MaxResults=100)['Qualifications']
quality_workers = [qual['WorkerId'] for qual in quals]

In [477]:
for worker_id in quality_workers:
    response_ass = client.associate_qualification_with_worker(
            QualificationTypeId='3BERVBF4P91CXVY7ZYVLFJAE0K7YPS',
            WorkerId=worker_id,
            IntegerValue=1,
            SendNotification=True
        )
    print(response_ass)

{'ResponseMetadata': {'RequestId': '86c3d9e0-a853-4e7a-85c3-69b120604111', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '86c3d9e0-a853-4e7a-85c3-69b120604111', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:54:30 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'adcd4820-d295-4561-b61b-f4c550d8c5cc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'adcd4820-d295-4561-b61b-f4c550d8c5cc', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:54:30 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '093690d6-b91e-42f5-8e82-4645706270cc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '093690d6-b91e-42f5-8e82-4645706270cc', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:54:32 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'd50f76b7-b332-4637-8040-8eab589619b3', 'HTTPSt

{'ResponseMetadata': {'RequestId': 'b358a067-ca1f-479c-b8c3-1924fa839c60', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b358a067-ca1f-479c-b8c3-1924fa839c60', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:54:36 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'e857915f-6f6f-498b-b75a-af8742f2bf30', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e857915f-6f6f-498b-b75a-af8742f2bf30', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:54:37 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '3caf36c6-100d-46e4-980e-36fe84e441a1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3caf36c6-100d-46e4-980e-36fe84e441a1', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:54:37 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'a9911614-874a-4f48-959b-b6b9edff97c2', 'HTTPSt

In [479]:
old_batch = ['A1UKZRDH1T231M',  'A70L26UXLTGLC',
       'A34DYM8J0X5VK', 'A1815TOQ1PZ7JH']
for worker in old_batch:
    print(worker in quality_workers)

True
True
True
True


In [480]:
for worker_id in old_batch:
    response_ass = client.disassociate_qualification_from_worker(
            QualificationTypeId='3BERVBF4P91CXVY7ZYVLFJAE0K7YPS',
            WorkerId=worker_id,
            Reason='Already annotated old batch.'
        )
    print(response_ass)

{'ResponseMetadata': {'RequestId': '52a70f68-5603-40df-b737-124925aa06ac', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '52a70f68-5603-40df-b737-124925aa06ac', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:55:23 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '0b5afa2d-07d4-4b69-be41-a0798b3ff788', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0b5afa2d-07d4-4b69-be41-a0798b3ff788', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:55:23 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'c622a7b2-583c-436e-be46-ae5c95d3afa3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c622a7b2-583c-436e-be46-ae5c95d3afa3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 22 Mar 2020 11:55:23 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '6e92b88b-8047-4c2f-8246-29a8c087808c', 'HTTPSt

### Qual 2

In [ ]:
response = client.create_qualification_type(
    Name='Re-annotate old HIT2',
    Keywords='pilot2',
    Description='Re-annotate old HIT2',
    QualificationTypeStatus='Active',
    AutoGranted=False
)
print(response)

In [426]:
for worker_id in quality_workers:
    response_ass = client.associate_qualification_with_worker(
            QualificationTypeId='3PHD6OHA876NG3N038H08WPJK8Q9WA',
            WorkerId=worker_id,
            IntegerValue=1,
            SendNotification=True
        )
    print(response_ass)

{'ResponseMetadata': {'RequestId': 'dd3d69ad-9d2f-4a9f-87bb-2a53f200c9a7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'dd3d69ad-9d2f-4a9f-87bb-2a53f200c9a7', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:26:18 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '98441ebf-9f6a-4ff2-9d91-e51105d3f6e6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '98441ebf-9f6a-4ff2-9d91-e51105d3f6e6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:26:18 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'f83af5aa-3647-4c05-96b5-7906a69cd592', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f83af5aa-3647-4c05-96b5-7906a69cd592', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:26:18 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '3751702b-2f13-45b8-9376-269f1204f9d1', 'HTTPSt

{'ResponseMetadata': {'RequestId': '0bd1626f-dc78-485e-8e05-d3cdda3f59ea', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0bd1626f-dc78-485e-8e05-d3cdda3f59ea', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:26:22 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '8fb5831a-412d-4451-af29-0cec2aa06bec', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8fb5831a-412d-4451-af29-0cec2aa06bec', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:26:22 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '37985578-945c-48c0-b07b-84fa9b3da0f3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '37985578-945c-48c0-b07b-84fa9b3da0f3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:26:22 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '1c12b8dd-b728-4ec0-8964-d35091d76456', 'HTTPSt

In [429]:
old_batch = ['A1815TOQ1PZ7JH',
 'A2ML4TEAJTDQD0',
 'A30HUZHJBOX1LK',
 'A30RTQC0LLY19K',
 'A3PL7JEMIPFHI9',
 'AIS4X2NYY3V1T']
for w in old_batch:
    print(w in quality_workers)

True
True
True
True
True
True


In [430]:
for worker_id in old_batch:
    response_ass = client.disassociate_qualification_from_worker(
            QualificationTypeId='3PHD6OHA876NG3N038H08WPJK8Q9WA',
            WorkerId=worker_id,
            Reason='Already annotated old batch.'
        )
    print(response_ass)

{'ResponseMetadata': {'RequestId': '02412213-4ef5-443a-907c-0e564ddc0d58', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '02412213-4ef5-443a-907c-0e564ddc0d58', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:27:19 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'd6fa438c-6f67-4d6f-ab8d-6f5febc8126e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd6fa438c-6f67-4d6f-ab8d-6f5febc8126e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:27:19 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '79b0a4c3-9b07-4056-9cd2-bfb5729fc5f4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '79b0a4c3-9b07-4056-9cd2-bfb5729fc5f4', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:27:19 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '2dab9bae-9d75-4ccf-82b9-d41b5d4a2431', 'HTTPSt

### Qual 3

In [219]:
response = client.create_qualification_type(
    Name='Re-annotate old HIT 3',
    Keywords='pilot',
    Description='Re-annotate old HIT 3',
    QualificationTypeStatus='Active',
    AutoGranted=False
)
print(response)

{'QualificationType': {'QualificationTypeId': '3O3C52ZQESTNOSOAHFVROOKMX83HSU', 'CreationTime': datetime.datetime(2020, 3, 19, 6, 55, 58, tzinfo=tzlocal()), 'Name': 'Re-annotate old HIT 3', 'Description': 'Re-annotate old HIT 3', 'Keywords': 'pilot', 'QualificationTypeStatus': 'Active', 'IsRequestable': True, 'AutoGranted': False}, 'ResponseMetadata': {'RequestId': 'cbd003d5-f00a-4172-b165-429a3b701fe7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cbd003d5-f00a-4172-b165-429a3b701fe7', 'content-type': 'application/x-amz-json-1.1', 'content-length': '271', 'date': 'Thu, 19 Mar 2020 11:55:57 GMT'}, 'RetryAttempts': 0}}


In [369]:
for worker_id in quality_workers:
    response_ass = client.associate_qualification_with_worker(
            QualificationTypeId='3O3C52ZQESTNOSOAHFVROOKMX83HSU',
            WorkerId=worker_id,
            IntegerValue=1,
            SendNotification=True
        )
    print(response_ass)

{'ResponseMetadata': {'RequestId': '727a5169-b9d9-4d99-9c40-c8116c55896b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '727a5169-b9d9-4d99-9c40-c8116c55896b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Fri, 20 Mar 2020 21:28:36 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '0ed6f371-640c-4004-b0cf-c8faa4dae153', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0ed6f371-640c-4004-b0cf-c8faa4dae153', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Fri, 20 Mar 2020 21:28:36 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'e08c0a20-b112-4bef-9b87-b59bd9d47957', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e08c0a20-b112-4bef-9b87-b59bd9d47957', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Fri, 20 Mar 2020 21:28:37 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '3f522229-097c-4c21-b1a7-9422e9de54e5', 'HTTPSt

{'ResponseMetadata': {'RequestId': '4d9edced-8ba7-4b9c-9969-e9a1877cee48', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4d9edced-8ba7-4b9c-9969-e9a1877cee48', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Fri, 20 Mar 2020 21:28:40 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '4180aa77-aa88-4621-aae9-a0b203bb9de4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4180aa77-aa88-4621-aae9-a0b203bb9de4', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Fri, 20 Mar 2020 21:28:40 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '88443a98-1660-45cf-89e9-0281627b59af', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '88443a98-1660-45cf-89e9-0281627b59af', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Fri, 20 Mar 2020 21:28:41 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '356dd565-817f-4cd6-9e96-e17ef07d7c72', 'HTTPSt

In [427]:
old_batch = ['A1815TOQ1PZ7JH',
 'A2ML4TEAJTDQD0',
 'A30HUZHJBOX1LK',
 'A30RTQC0LLY19K',
 'A3PL7JEMIPFHI9',
 'AIS4X2NYY3V1T']
for worker in old_batch:
    print(worker in quality_workers)

True
True
True
True
True
True


In [428]:
for worker_id in old_batch:
    response_ass = client.disassociate_qualification_from_worker(
            QualificationTypeId='3O3C52ZQESTNOSOAHFVROOKMX83HSU',
            WorkerId=worker_id,
            Reason='Already annotated old batch.'
        )
    print(response_ass)

{'ResponseMetadata': {'RequestId': '4303bce8-0132-4175-b234-a14c44f766f9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4303bce8-0132-4175-b234-a14c44f766f9', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sat, 21 Mar 2020 20:26:42 GMT'}, 'RetryAttempts': 0}}


RequestError: An error occurred (RequestError) when calling the DisassociateQualificationFromWorker operation: This operation can be called with a status of: Granted (1584822403070)

### Create qualification for HIGH QUALITY WORKERS

In [ ]:
response = client.create_qualification_type(
    Name='High quality workers for Intent task',
    Keywords='pilot',
    Description='Workers who gave high quality annotations and explanations in pilot of Intent task.',
    QualificationTypeStatus='Active',
    AutoGranted=False
)
print(response)

In [37]:
for worker_id in ['A30RTQC0LLY19K', 'A1815TOQ1PZ7JH']:
    response_ass = client.associate_qualification_with_worker(
            QualificationTypeId='3D39V6MAT4NEO0YLHPBLW6N58MO8C9',
            WorkerId=worker_id,
            IntegerValue=1,
            SendNotification=True
        )
    print(response_ass)

{'ResponseMetadata': {'RequestId': '77421ce9-6c5d-4385-ae9b-03b98c3b4caa', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '77421ce9-6c5d-4385-ae9b-03b98c3b4caa', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 15 Mar 2020 00:35:31 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '53e3cb33-25e2-4b1c-af76-4e654e835abc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '53e3cb33-25e2-4b1c-af76-4e654e835abc', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Sun, 15 Mar 2020 00:35:31 GMT'}, 'RetryAttempts': 0}}


In [466]:
#for worker_id in low_workers:
worker_id = 'A3L2VS3998R77L'
response_ass = client.disassociate_qualification_from_worker(
        QualificationTypeId='3D39V6MAT4NEO0YLHPBLW6N58MO8C9',
        WorkerId=worker_id,
        Reason='Thank you for all your work on this task. We are now focusing on a subset of the original workers.'
    )
print(response_ass)

RequestError: An error occurred (RequestError) when calling the DisassociateQualificationFromWorker operation: This operation can be called with a status of: Granted (1584877050069)

In [ ]:
with open('qualified_workers_03-05-2020.txt', 'w') as f:
    for quality_worker in quality_workers:
        f.write("%s\n" % quality_worker)

In [20]:
worker_id = ['A30HUZHJBOX1LK', 'A30RTQC0LLY19K', 'A34DYM8J0X5VK']
response = client.notify_workers(
    Subject="Feedback for Witness Intent Task: Explanations for partial answers or answers to different questions",
    MessageText="Thank you for all your work on this task. We have some feedback on your explanations when you judge the witness is giving a partial answer or answering a different question. Please briefly explain what part of the answer was not answered, or what question was actually answered. On a side note, we understand you may no longer be able to complete HITs due to the Coronavirus impacts.",
    WorkerIds=worker_id
)
print(response)

{'NotifyWorkersFailureStatuses': [], 'ResponseMetadata': {'RequestId': '02c19cb3-4f9c-47ef-9b49-1d2ebe0db0a6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '02c19cb3-4f9c-47ef-9b49-1d2ebe0db0a6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '35', 'date': 'Sat, 14 Mar 2020 22:52:21 GMT'}, 'RetryAttempts': 0}}


In [465]:
worker_id = ['A2ML4TEAJTDQD0']
response = client.notify_workers(
    Subject="Feedback for Witness Intent Task: How to label responses when the witness refuses to answer or says they don't know",
    MessageText="Thank you for all your work on this task. We have some feedback on your choice of witness intent. When the witness says they can't answer the question or they don't know the answer, please choose the last radio button 'says they don't understand/don't know/can't answer or asks...'. For example, when a witness responds 'I can't comment on that.', you should choose this radio button. (Of course, whether you believe they are lying or sincere is your subjective judgement). Please review the instructions for more examples. Thank you again! On a side note, we understand you may no longer be able to complete HITs on the platform due to the Coronavirus impacts.",
    WorkerIds=worker_id
)
print(response)

{'NotifyWorkersFailureStatuses': [], 'ResponseMetadata': {'RequestId': '56785718-9396-4d82-bd71-a2131a44be59', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '56785718-9396-4d82-bd71-a2131a44be59', 'content-type': 'application/x-amz-json-1.1', 'content-length': '35', 'date': 'Sun, 22 Mar 2020 11:32:28 GMT'}, 'RetryAttempts': 0}}


### grant bonus

In [ ]:
bonus_worker_infos=[
    ['3MH9DQ757XBZ35XPL4WSUZ1WJ13UG1', 'A2A08F9GQ2IFG7'],
       ['3KKG4CDWKJXL7XG2SSZ0PEHFG9149A', 'A1BIN5R8FF4FQR'],
       ['33PPO7FECWERIUSYPX665BAOCCCDIA', 'A105ZUIAGMN8SW'],
       ['3L70J4KAZHLCCC90D9ETODM5Z5IADG', 'A17VJGXGBNEL9E'],
       ['37UEWGM5HU7AJYF5ZRGKI1A6ABZR19', 'A2Y9ZNZ0F24GHB'],
       ['34Q075JO1YCMJ4V65EX5Y59J1ME010', 'A1AJ2G7JXXJ8UJ'],
       ['352YTHGROWCEM3YLQ4NS2VF3ZOT4HO', 'A3EOBVZ13L64O5'],
       ['3N8OEVH1FSPSIDUQD1XR02I9S29OOA', 'A3QBYYDQGEFHA8'],
       ['3HL8HNGX460V5DDRC9VJGIE65179F4', 'A3PL7JEMIPFHI9'],
       ['3YMU66OBIO7UYJR5L5PTBKC6PT4HGQ', 'A3PFEE6DHRZEL6'],
       ['38YMOXR4MVYAYGXO4R270W94A3OW6D', 'AIS4X2NYY3V1T'],
       ['3P59JYT76MJUOYATMKLTNZLTQ6VT2Y', 'AGI56GIPBZQ4Z'],
       ['3WQQ9FUS6BT5ERI12F39XWSA6ZNB8T', 'AGI56GIPBZQ4Z'],
       ['30H4UDGLT3HM3ALFFLVTXYYU76TPME', 'AU01P6YB9J5JX']
]

In [ ]:
for bonus_worker_info in bonus_worker_infos:
    response_grant = client.send_bonus(
        AssignmentId=bonus_worker_info[0],
        WorkerId=bonus_worker_info[1],
        Reason='Adherence to instructions and meaningful explanations. Thank you for your careful work and be on the lookout for new HITs soon.',
        BonusAmount='0.60'
    )
    print(response_grant)

In [ ]:
for bonus_worker_info in low_paid_worker_infos:
    response = client.approve_assignment(AssignmentId=bonus_worker_info[0])
    print(response)

In [ ]:
for bonus_worker_info in low_paid_worker_infos:
    response_grant = client.send_bonus(
        AssignmentId=bonus_worker_info[0],
        WorkerId=bonus_worker_info[1],
        Reason='Insufficient payment for particularly long HIT. Our previous payment was based on time estimates of congressional hearings with considerably shorted questions and responses. We recognize this particular hearing is much longer and the estimated compensation was not adequate.',
        BonusAmount='0.50'
    )
    print(response_grant)

In [ ]:
### Create qualification for SINGLE Hit

In [ ]:
response = client.create_qualification_type(
    Name='Facebook hearing',
    Keywords='pilot',
    Description='Qualified workers to annotate Facebook hearing',
    QualificationTypeStatus='Active',
    AutoGranted=False
)
print(response)

In [ ]:
workers_not_annotated = ['A1FGKIKJYSL1MI', 'AB8BRFF4ZTZ40', 'AZ5ZYUCAQ0XDL',
       'A36MEZ1Y8KLTGY', 'A2XS5DMRS7MG9N', 'AP9M4LGBH8FYM',
       'A2YQ388TEOCOL1', 'A1UKZRDH1T231M', 'A34DYM8J0X5VK',
       'A1OBB3PWYWK9KK', 'A1QJV5CO3Z4DHK', 'A6G30K3SIWKZ9',
       'A1EE6KAV0P12KQ', 'A1NIKHUA79SWGD', 'A1N53V41FMCR7Q',
       'A1O9PAK6THJ9Y7', 'A3L2VS3998R77L', 'A70L26UXLTGLC',
       'A1JW11G1QUNMGN', 'A1QXBYE81CP9N9', 'A2P8V5SKYLL5I4',
       'A1815TOQ1PZ7JH', 'A2F2DDH12YU4AK', 'A1CY7IOJ9YH136']

In [ ]:
for worker_id in workers_not_annotated:
    response_ass = client.associate_qualification_with_worker(
            QualificationTypeId='31TD7GZTII8UHLWBGKKP4QEQW9COZP',
            WorkerId=worker_id,
            IntegerValue=1,
            SendNotification=True
        )
    print(response_ass)

In [ ]:
for worker_id in workers_not_annotated:
    response_ass = client.associate_qualification_with_worker(
            QualificationTypeId='31TD7GZTII8UHLWBGKKP4QEQW9COZP',
            WorkerId=worker_id,
            IntegerValue=1,
            SendNotification=True
        )
    print(response_ass)

### Reject Workers

In [ ]:
rejected_worker_infos = [
       ['3H0W84IWBL193Z9HBFBZPLT0ZK2RER', 'A7K76OHR88ZI9'],
       ['3P59JYT76MJUOYATMKLTNZLTQ482TG', 'AXV2CA5TY066N'],
       ['3VHP9MDGROJX3BKUOT5AI8FEBODFCY', 'A3EDGV5ERV8G9O']
]

In [ ]:
for rejected_worker_info in rejected_worker_infos:
    response_rej = client.reject_assignment(
            AssignmentId=rejected_worker_info[0],
            RequesterFeedback='Required explanations were non-sensical.'
        )
    print(response_rej)